In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.8 MB/s eta 0:00:00


In [ ]:
!cp ./drive/MyDrive/chunks.zip ./chunks.zip

In [ ]:
!unzip chunks.zip

Archive:  chunks.zip
   creating: chunks/
  inflating: chunks/chunk_2_processed.csv  
  inflating: chunks/chunk_6_processed.csv  
  inflating: chunks/chunk_1_processed.csv  
  inflating: chunks/chunk_5_processed.csv  
  inflating: chunks/chunk_1.csv      
  inflating: chunks/chunk_2.csv      
  inflating: chunks/chunk_4_processed.csv  
  inflating: chunks/chunk_3.csv      
  inflating: chunks/chunk_6.csv      
  inflating: chunks/chunk_3_processed.csv  
  inflating: chunks/chunk_4.csv      
  inflating: chunks/chunk_5.csv      


In [ ]:
!mkdir ./drive/MyDrive/vector_storages

In [ ]:
import pandas as pd

from tqdm.auto import tqdm
from backend.AI_services.ai_services.vector_storage import VectorStorage

In [ ]:
def process_text(text):
    return "".join([char for char in text.lower() if char.isalnum() or char.isspace()]).strip()

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('intfloat/e5-base-v2', device="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
files = [
    "./chunks/chunk_1_processed.csv",
    "./chunks/chunk_2_processed.csv",
    "./chunks/chunk_3_processed.csv",
    "./chunks/chunk_4_processed.csv",
    "./chunks/chunk_5_processed.csv",
    "./chunks/chunk_6_processed.csv",
]

In [ ]:
with tqdm(total=len(files), desc="building storages") as pbar:
    for file in files:
        data = pd.read_csv(file)
        data.text = data.text.apply(process_text)
        storage = VectorStorage(
            dim=model.get_sentence_embedding_dimension(),
            embedder=model.encode,
        )
        metadata = [{"text": text} for text in data["text"]]
        storage.add_documents(
            ids=data.index.tolist(),
            texts=data["text"].tolist(),
            metadata=metadata
        )
        filename = file.split("/")[-1].split(".")[0]
        storage.save(f"./drive/MyDrive/vector_storages/storage-{filename}")
        pbar.update(1)

building storages:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8182 [00:00<?, ?it/s]

  0%|          | 0/8603 [00:00<?, ?it/s]

  0%|          | 0/7891 [00:00<?, ?it/s]

  0%|          | 0/7183 [00:00<?, ?it/s]

  0%|          | 0/6487 [00:00<?, ?it/s]

  0%|          | 0/5803 [00:00<?, ?it/s]